<a href="https://colab.research.google.com/github/srikanthrc/running-to-stand-still/blob/master/_notebooks/2020-03-21-covid19_overview.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# My COVID-19 Overview
> Tracking coronavirus total cases, deaths and new cases by country.  Additionally, a detailed view is provided for the US by state.

- author: Srikanth Chinmay (inspired by Pratap Vardhan @ http://covid19dashboards.com) 
- categories: [covid-19]
- image: images/covid-overview.png
- permalink: /covid-overview/
- toc: true
- comments: false


# Part One: Statistics By Country

In [1]:
#hide
print('''
Example of using jupyter notebook, pandas (data transformations), jinja2 (html, visual)
to create visual dashboards with fastpages
You see also the live version on https://gramener.com/enumter/covid19/
''')


Example of using jupyter notebook, pandas (data transformations), jinja2 (html, visual)
to create visual dashboards with fastpages
You see also the live version on https://gramener.com/enumter/covid19/



In [0]:
#hide
import numpy as np
import pandas as pd
from jinja2 import Template
from IPython.display import HTML

In [0]:
#hide

# FETCH
import getpass
base_url = 'https://raw.githubusercontent.com/pratapvardhan/notebooks/master/covid19/'
base_url = '' if (getpass.getuser() == 'Pratap Vardhan') else base_url
paths = {
    'mapping': base_url + 'mapping_countries.csv',
    'overview': base_url + 'overview.tpl'
}

def get_mappings(url):
    df = pd.read_csv(url)
    return {
        'df': df,
        'replace.country': dict(df.dropna(subset=['Name']).set_index('Country')['Name']),
        'map.continent': dict(df.set_index('Name')['Continent'])
    }

mapping = get_mappings(paths['mapping'])

def get_template(path):
    from urllib.parse import urlparse
    if bool(urlparse(path).netloc):
        from urllib.request import urlopen
        return urlopen(path).read().decode('utf8')
    return open(path).read()

def get_frame(name):
    url = (
        'https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/'
        f'csse_covid_19_time_series/time_series_19-covid-{name}.csv')
    df = pd.read_csv(url)
    # rename countries
    df['Country/Region'] = df['Country/Region'].replace(mapping['replace.country'])
    return df

def get_dates(df):
    dt_cols = df.columns[~df.columns.isin(['Province/State', 'Country/Region', 'Lat', 'Long'])]
    LAST_DATE_I = -1
    # sometimes last column may be empty, then go backwards
    for i in range(-1, -len(dt_cols), -1):
        if not df[dt_cols[i]].fillna(0).eq(0).all():
            LAST_DATE_I = i
            break
    return LAST_DATE_I, dt_cols

In [0]:
#hide
COL_REGION = 'Country/Region'
# Confirmed, Recovered, Deaths
df = get_frame('Confirmed')
# dft_: timeseries, dfc_: today country agg
dft_cases = df
dft_deaths = get_frame('Deaths')
dft_recovered = get_frame('Recovered')
LAST_DATE_I, dt_cols = get_dates(df)

dt_today = dt_cols[LAST_DATE_I]
dt_ago = dt_cols[LAST_DATE_I-1]

dfc_cases = dft_cases.groupby(COL_REGION)[dt_today].sum()
dfc_deaths = dft_deaths.groupby(COL_REGION)[dt_today].sum()
dfc_recovered = dft_recovered.groupby(COL_REGION)[dt_today].sum()
dfp_cases = dft_cases.groupby(COL_REGION)[dt_ago].sum()
dfp_deaths = dft_deaths.groupby(COL_REGION)[dt_ago].sum()
dfp_recovered = dft_recovered.groupby(COL_REGION)[dt_ago].sum()

In [5]:
#hide
df_table = (pd.DataFrame(dict(
    Cases=dfc_cases, Deaths=dfc_deaths, Recovered=dfc_recovered,
    PCases=dfp_cases, PDeaths=dfp_deaths, PRecovered=dfp_recovered))
             .sort_values(by=['Cases', 'Deaths'], ascending=[False, False])
             .reset_index())
for c in 'Cases, Deaths, Recovered'.split(', '):
    df_table[f'{c} (+)'] = (df_table[c] - df_table[f'P{c}']).clip(0)  # DATA BUG
df_table['Fatality Rate'] = (100 * df_table['Deaths'] / df_table['Cases']).round(1)
df_table['Continent'] = df_table['Country/Region'].map(mapping['map.continent'])
df_table.head(15)

Country/Region  Cases  Deaths  ...  Recovered (+)  Fatality Rate      Continent
0            China  81305    3259  ...            591            4.0           Asia
1            Italy  53578    4825  ...           1632            9.0         Europe
2               US  25489     307  ...              0            1.2  North America
3            Spain  25374    1375  ...            537            5.4         Europe
4          Germany  22213      84  ...             53            0.4         Europe
5             Iran  20610    1556  ...            890            7.5           Asia
6           France  14431     562  ...              0            3.9         Europe
7      South Korea   8799     102  ...              0            1.2           Asia
8      Switzerland   6575      75  ...              0            1.1         Europe
9   United Kingdom   5067     234  ...              0            4.6         Europe
10     Netherlands   3640     137  ...              0            3.8         Europe
11         Belgium   2815      67  ...            262            2.4         Europe
12         Austria   2814       8  ...              0            0.3         Europe
13          Norway   2118       7  ...              0            0.3         Europe
14          Sweden   1763      20  ...              0            1.1         Europe

[15 rows x 12 columns]

In [6]:
#hide
# world, china, europe, us
metrics = ['Cases', 'Deaths', 'Recovered', 'Cases (+)', 'Deaths (+)', 'Recovered (+)']
s_china = df_table[df_table['Country/Region'].eq('China')][metrics].sum().add_prefix('China ')
s_us = df_table[df_table['Country/Region'].eq('US')][metrics].sum().add_prefix('US ')
s_eu = df_table[df_table['Continent'].eq('Europe')][metrics].sum().add_prefix('EU ')
summary = {'updated': pd.to_datetime(dt_today), 'since': pd.to_datetime(dt_ago)}
summary = {**summary, **df_table[metrics].sum(), **s_china, **s_us, **s_eu}
summary

{'Cases': 304524,
 'Cases (+)': 32358,
 'China Cases': 81305,
 'China Cases (+)': 55,
 'China Deaths': 3259,
 'China Deaths (+)': 6,
 'China Recovered': 71857,
 'China Recovered (+)': 591,
 'Deaths': 12973,
 'Deaths (+)': 1678,
 'EU Cases': 150803,
 'EU Cases (+)': 21481,
 'EU Deaths': 7490,
 'EU Deaths (+)': 1429,
 'EU Recovered': 8996,
 'EU Recovered (+)': 2554,
 'Recovered': 91499,
 'Recovered (+)': 4245,
 'US Cases': 25489,
 'US Cases (+)': 6389,
 'US Deaths': 307,
 'US Deaths (+)': 63,
 'US Recovered': 0,
 'US Recovered (+)': 0,
 'since': Timestamp('2020-03-20 00:00:00'),
 'updated': Timestamp('2020-03-21 00:00:00')}

In [7]:
#hide
dft_ct_cases = dft_cases.groupby(COL_REGION)[dt_cols].sum()
dft_ct_new_cases = dft_ct_cases.diff(axis=1).fillna(0).astype(int)
dft_ct_new_cases.head()

1/22/20  1/23/20  1/24/20  ...  3/19/20  3/20/20  3/21/20
Country/Region                             ...                           
Afghanistan           0        0        0  ...        0        2        0
Albania               0        0        0  ...        5        6        6
Algeria               0        0        0  ...       13        3       49
Andorra               0        0        0  ...       14       22       13
Angola                0        0        0  ...        0        1        1

[5 rows x 60 columns]

In [8]:
#hide_input
template = Template(get_template(paths['overview']))
html = template.render(
    D=summary, table=df_table.head(20),  # REMOVE .head(20) to see all values
    newcases=dft_ct_new_cases.loc[:, dt_cols[LAST_DATE_I-50]:dt_cols[LAST_DATE_I]],
    np=np, pd=pd, enumerate=enumerate)
HTML(f'<div>{html}</div>')

# Part Two: United States, by State

In [0]:
#hide

# FETCH
import getpass
base_url = 'https://raw.githubusercontent.com/pratapvardhan/notebooks/master/covid19/'
base_url = '' if (getpass.getuser() == 'Pratap Vardhan') else base_url
paths = {
    'mapping': base_url + 'mapping_countries.csv',
    'overview': base_url + 'overview.tpl'
}

def get_mappings(url):
    df = pd.read_csv(url)
    return {
        'df': df,
        'replace.country': dict(df.dropna(subset=['Name']).set_index('Country')['Name']),
        'map.continent': dict(df.set_index('Name')['Continent'])
    }

mapping = get_mappings(paths['mapping'])

def get_template(path):
    from urllib.parse import urlparse
    if bool(urlparse(path).netloc):
        from urllib.request import urlopen
        return urlopen(path).read().decode('utf8')
    return open(path).read()

def get_frame(name):
    url = (
        'https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/'
        f'csse_covid_19_time_series/time_series_19-covid-{name}.csv')
    df = pd.read_csv(url)
    # rename countries
    df['Country/Region'] = df['Country/Region'].replace(mapping['replace.country'])
    return df

def get_dates(df):
    dt_cols = df.columns[~df.columns.isin(['Province/State', 'Country/Region', 'Lat', 'Long'])]
    LAST_DATE_I = -1
    # sometimes last column may be empty, then go backwards
    for i in range(-1, -len(dt_cols), -1):
        if not df[dt_cols[i]].fillna(0).eq(0).all():
            LAST_DATE_I = i
            break
    return LAST_DATE_I, dt_cols

In [0]:
#hide
COL_REGION = 'Province/State'   
# Confirmed, Recovered, Deaths
US_POI = [
    'Alabama', 'Alaska', 'Arizona', 'Arkansas', 'California',
    'Colorado', 'Connecticut', 'Delaware', 'Diamond Princess',
    'District of Columbia', 'Florida', 'Georgia', 'Grand Princess',
    'Guam', 'Hawaii', 'Idaho', 'Illinois', 'Indiana', 'Iowa', 'Kansas',
    'Kentucky', 'Louisiana', 'Maine', 'Maryland', 'Massachusetts',
    'Michigan', 'Minnesota', 'Mississippi', 'Missouri', 'Montana',
    'Nebraska', 'Nevada', 'New Hampshire', 'New Jersey', 'New Mexico',
    'New York', 'North Carolina', 'North Dakota', 'Ohio', 'Oklahoma',
    'Oregon', 'Pennsylvania', 'Puerto Rico', 'Rhode Island',
    'South Carolina', 'South Dakota', 'Tennessee', 'Texas', 'Utah',
    'Vermont', 'Virgin Islands', 'Virginia', 'Washington',
    'West Virginia', 'Wisconsin', 'Wyoming']

filter_us = lambda d: d[d['Country/Region'].eq('US') & d['Province/State'].isin(US_POI)]

df = get_frame('Confirmed')
# dft_: timeseries, dfc_: today country agg
dft_cases = df.pipe(filter_us)
dft_deaths = get_frame('Deaths').pipe(filter_us)
dft_recovered = get_frame('Recovered').pipe(filter_us)

LAST_DATE_I, dt_cols = get_dates(df)

dt_today = dt_cols[LAST_DATE_I]
dt_ago = dt_cols[LAST_DATE_I-1]

dfc_cases = dft_cases.groupby(COL_REGION)[dt_today].sum()
dfc_deaths = dft_deaths.groupby(COL_REGION)[dt_today].sum()
dfc_recovered = dft_recovered.groupby(COL_REGION)[dt_today].sum()
dfp_cases = dft_cases.groupby(COL_REGION)[dt_ago].sum()
dfp_deaths = dft_deaths.groupby(COL_REGION)[dt_ago].sum()
dfp_recovered = dft_recovered.groupby(COL_REGION)[dt_ago].sum()

In [11]:
#hide
df_table = (pd.DataFrame(dict(
    Cases=dfc_cases, Deaths=dfc_deaths, Recovered=dfc_recovered,
    PCases=dfp_cases, PDeaths=dfp_deaths, PRecovered=dfp_recovered))
             .sort_values(by=['Cases', 'Deaths'], ascending=[False, False])
             .reset_index())
for c in 'Cases, Deaths, Recovered'.split(', '):
    df_table[f'{c} (+)'] = (df_table[c] - df_table[f'P{c}']).clip(0)  # DATA BUG
df_table['Fatality Rate'] = (100 * df_table['Deaths'] / df_table['Cases']).round(1)
df_table.head(15)

Province/State  Cases  Deaths  ...  Deaths (+)  Recovered (+)  Fatality Rate
0        New York  11710      60  ...          18              0            0.5
1      Washington   1793      94  ...          11              0            5.2
2      California   1364      24  ...           1              0            1.8
3      New Jersey   1327      16  ...           5              0            1.2
4        Michigan    788       5  ...           2              0            0.6
5        Illinois    753       6  ...           1              0            0.8
6         Florida    659      13  ...           3              0            2.0
7       Louisiana    585      16  ...           2              0            2.7
8           Texas    581       5  ...           0              0            0.9
9   Massachusetts    525       1  ...           0              0            0.2
10        Georgia    507      14  ...           1              0            2.8
11   Pennsylvania    396       2  ...           1              0            0.5
12       Colorado    390       4  ...           0              0            1.0
13      Tennessee    371       1  ...           1              0            0.3
14      Wisconsin    282       4  ...           1              0            1.4

[15 rows x 11 columns]

In [12]:
#hide
# world, china, europe, us
metrics = ['Cases', 'Deaths', 'Recovered', 'Cases (+)', 'Deaths (+)', 'Recovered (+)']
s_ny = df_table[df_table[COL_REGION].eq('New York')][metrics].sum().add_prefix('NY ')
s_wa = df_table[df_table[COL_REGION].eq('Washington')][metrics].sum().add_prefix('WA ')
s_ca = df_table[df_table[COL_REGION].eq('California')][metrics].sum().add_prefix('CA ')
summary = {'updated': pd.to_datetime(dt_today), 'since': pd.to_datetime(dt_ago)}
summary = {**summary, **df_table[metrics].sum(), **s_ny, **s_wa, **s_ca}
summary

{'CA Cases': 1364,
 'CA Cases (+)': 187,
 'CA Deaths': 24,
 'CA Deaths (+)': 1,
 'CA Recovered': 0,
 'CA Recovered (+)': 0,
 'Cases': 25489,
 'Cases (+)': 6389,
 'Deaths': 307,
 'Deaths (+)': 63,
 'NY Cases': 11710,
 'NY Cases (+)': 3400,
 'NY Deaths': 60,
 'NY Deaths (+)': 18,
 'NY Recovered': 0,
 'NY Recovered (+)': 0,
 'Recovered': 0,
 'Recovered (+)': 0,
 'WA Cases': 1793,
 'WA Cases (+)': 269,
 'WA Deaths': 94,
 'WA Deaths (+)': 11,
 'WA Recovered': 0,
 'WA Recovered (+)': 0,
 'since': Timestamp('2020-03-20 00:00:00'),
 'updated': Timestamp('2020-03-21 00:00:00')}

In [13]:
#hide
dft_ct_cases = dft_cases.groupby(COL_REGION)[dt_cols].sum()
dft_ct_new_cases = dft_ct_cases.diff(axis=1).fillna(0).astype(int)
dft_ct_new_cases.head()

1/22/20  1/23/20  1/24/20  ...  3/19/20  3/20/20  3/21/20
Province/State                             ...                           
Alabama               0        0        0  ...       32        5       48
Alaska                0        0        0  ...        3        3        3
Arizona               0        0        0  ...       18       33       40
Arkansas              0        0        0  ...       29       34       26
California            0        0        0  ...      201      225      187

[5 rows x 60 columns]

In [14]:
#hide_input
template = Template(get_template(paths['overview']))
html = template.render(
    D=summary, table=df_table,
    newcases=dft_ct_new_cases.loc[:, dt_cols[LAST_DATE_I-15]:dt_cols[LAST_DATE_I]],
    COL_REGION=COL_REGION,
    KPI_CASE='US',
    KPIS_INFO=[
        {'title': 'New York', 'prefix': 'NY'},
        {'title': 'Washington', 'prefix': 'WA'},
        {'title': 'California', 'prefix': 'CA'}],
    LEGEND_DOMAIN=[5, 50, 500, np.inf],
    np=np, pd=pd, enumerate=enumerate)
HTML(f'<div>{html}</div>')

Visualizations by [Pratap Vardhan](https://twitter.com/PratapVardhan)[^1]

[^1]: Source: ["COVID-19 Data Repository by Johns Hopkins CSSE"](https://systems.jhu.edu/research/public-health/ncov/) [GitHub repository](https://github.com/CSSEGISandData/COVID-19). Link to [notebook](https://github.com/pratapvardhan/notebooks/blob/master/covid19/covid19-overview.ipynb), [orignal interactive](https://gramener.com/enumter/covid19/)